In [2]:
# ==============================================================================
#      VIRTUALFIT: FINAL COLAB BACKEND SERVER (Corrected)
# ==============================================================================
# This version removes the faulty debug check and relies on the model's native
# data loading, which is already proven to work with your file structure.
# ==============================================================================

# --- SEGMENT 0: SERVER & DEPENDENCY IMPORTS ---
print("--- SEGMENT 0: INSTALLING LIBRARIES & IMPORTS ---")
!pip install -q "fastapi[all]" pyngrok
import os
import shutil
import subprocess
import nest_asyncio
from google.colab import drive
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from fastapi.responses import FileResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
from pyngrok import ngrok

# --- SEGMENT 1: ENVIRONMENT SETUP ---
print("\n--- SEGMENT 1: SETTING UP THE ENVIRONMENT ---")
drive.mount('/content/drive')
PROJECT_BASE_DIR = '/content/drive/MyDrive/VirtualFIT_Models'
DRIVE_ASSETS_DIR = os.path.join(PROJECT_BASE_DIR, 'viton_hd_zips')
DRIVE_TEST_DATA_PATH = os.path.join(DRIVE_ASSETS_DIR, 'test')
DRIVE_CHECKPOINTS_ZIP_PATH = os.path.join(DRIVE_ASSETS_DIR, 'checkpoints.zip')
VITON_HD_CLONE_DIR = '/content/VITON_HD_repo'
VITON_HD_DATA_DIR = os.path.join(VITON_HD_CLONE_DIR, 'datasets')
VITON_HD_TEST_DIR = os.path.join(VITON_HD_DATA_DIR, 'test')
VITON_HD_CHECKPOINTS_DIR = os.path.join(VITON_HD_CLONE_DIR, 'checkpoints')

os.chdir('/content')
!rm -rf {VITON_HD_CLONE_DIR}
!git clone -q https://github.com/shadow2496/VITON-HD.git {VITON_HD_CLONE_DIR}
os.chdir(VITON_HD_CLONE_DIR)

os.makedirs(VITON_HD_TEST_DIR, exist_ok=True)
os.makedirs(VITON_HD_CHECKPOINTS_DIR, exist_ok=True)

print(f"\nCopying base assets from: {DRIVE_ASSETS_DIR}")
if os.path.isdir(DRIVE_TEST_DATA_PATH):
    # This command copies all your subdirectories with their existing names
    !cp -r "{DRIVE_TEST_DATA_PATH}"/* "{VITON_HD_TEST_DIR}/"
if os.path.exists(DRIVE_CHECKPOINTS_ZIP_PATH):
    !cp "{DRIVE_CHECKPOINTS_ZIP_PATH}" "{VITON_HD_CHECKPOINTS_DIR}/"
    !unzip -o -q "{os.path.join(VITON_HD_CHECKPOINTS_DIR, 'checkpoints.zip')}" -d "{VITON_HD_CHECKPOINTS_DIR}/"
    if os.path.exists(os.path.join(VITON_HD_CHECKPOINTS_DIR, 'checkpoints')):
        !mv {VITON_HD_CHECKPOINTS_DIR}/checkpoints/* {VITON_HD_CHECKPOINTS_DIR}/ && rm -rf {VITON_HD_CHECKPOINTS_DIR}/checkpoints

# --- SEGMENT 2: PATCH SCRIPT & INSTALL DEPENDENCIES ---
print("\n--- SEGMENT 2: PATCHING SCRIPT & INSTALLING DEPENDENCIES ---")
!pip install -q torch torchvision opencv-python torchgeometry
TEST_PY_PATH = os.path.join(VITON_HD_CLONE_DIR, 'test.py')
corrected_test_py_content = r"""
import argparse, os, torch; from torch import nn; from torch.nn import functional as F; import torchgeometry as tgm
from datasets import VITONDataset, VITONDataLoader; from networks import SegGenerator, GMM, ALIASGenerator; from utils import gen_noise, load_checkpoint, save_images
def get_opt():
    parser = argparse.ArgumentParser(); parser.add_argument('--name', type=str, required=True); parser.add_argument('-b', '--batch_size', type=int, default=1); parser.add_argument('-j', '--workers', type=int, default=1)
    parser.add_argument('--load_height', type=int, default=1024); parser.add_argument('--load_width', type=int, default=768); parser.add_argument('--shuffle', action='store_true')
    parser.add_argument('--dataset_dir', type=str, default='./datasets/'); parser.add_argument('--dataset_mode', type=str, default='test'); parser.add_argument('--dataset_list', type=str, default='test_pairs.txt')
    parser.add_argument('--checkpoint_dir', type=str, default='./checkpoints/'); parser.add_argument('--save_dir', type=str, default='./results/'); parser.add_argument('--display_freq', type=int, default=1)
    parser.add_argument('--seg_checkpoint', type=str, default='seg_final.pth'); parser.add_argument('--gmm_checkpoint', type=str, default='gmm_final.pth'); parser.add_argument('--alias_checkpoint', type=str, default='alias_final.pth')
    parser.add_argument('--semantic_nc', type=int, default=13); parser.add_argument('--init_type', choices=['normal', 'xavier', 'xavier_uniform', 'kaiming', 'orthogonal', 'none'], default='xavier')
    parser.add_argument('--init_variance', type=float, default=0.02); parser.add_argument('--grid_size', type=int, default=5); parser.add_argument('--norm_G', type=str, default='spectralaliasinstance')
    parser.add_argument('--ngf', type=int, default=64); parser.add_argument('--num_upsampling_layers', choices=['normal', 'more', 'most'], default='most'); opt = parser.parse_args(); return opt
def test(opt, seg, gmm, alias, device):
    up = nn.Upsample(size=(opt.load_height, opt.load_width), mode='bilinear'); gauss = tgm.image.GaussianBlur((15, 15), (3, 3)).to(device)
    test_dataset = VITONDataset(opt); test_loader = VITONDataLoader(opt, test_dataset)
    with torch.no_grad():
        for i, inputs in enumerate(test_loader.data_loader):
            img_names = inputs['img_name']; c_names = inputs['c_name']['unpaired']; img_agnostic = inputs['img_agnostic'].to(device)
            parse_agnostic = inputs['parse_agnostic'].to(device); pose = inputs['pose'].to(device); c = inputs['cloth']['unpaired'].to(device); cm = inputs['cloth_mask']['unpaired'].to(device)
            parse_agnostic_down = F.interpolate(parse_agnostic, size=(256, 192), mode='bilinear'); pose_down = F.interpolate(pose, size=(256, 192), mode='bilinear')
            c_masked_down = F.interpolate(c * cm, size=(256, 192), mode='bilinear'); cm_down = F.interpolate(cm, size=(256, 192), mode='bilinear')
            seg_input = torch.cat((cm_down, c_masked_down, parse_agnostic_down, pose_down, gen_noise(cm_down.size()).to(device)), dim=1)
            parse_pred_down = seg(seg_input); parse_pred = gauss(up(parse_pred_down)); parse_pred = parse_pred.argmax(dim=1)[:, None]
            parse_old = torch.zeros(parse_pred.size(0), 13, opt.load_height, opt.load_width, dtype=torch.float).to(device); parse_old.scatter_(1, parse_pred, 1.0)
            labels = {0:['background',[0]], 1:['paste',[2,4,7,8,9,10,11]], 2:['upper',[3]], 3:['hair',[1]], 4:['left_arm',[5]], 5:['right_arm',[6]], 6:['noise',[12]]}
            parse = torch.zeros(parse_pred.size(0), 7, opt.load_height, opt.load_width, dtype=torch.float).to(device)
            for j in range(len(labels)):
                for label in labels[j][1]: parse[:, j] += parse_old[:, label]
            agnostic_gmm = F.interpolate(img_agnostic, size=(256, 192), mode='nearest'); parse_cloth_gmm = F.interpolate(parse[:, 2:3], size=(256, 192), mode='nearest')
            pose_gmm = F.interpolate(pose, size=(256, 192), mode='nearest'); c_gmm = F.interpolate(c, size=(256, 192), mode='nearest'); gmm_input = torch.cat((parse_cloth_gmm, pose_gmm, agnostic_gmm), dim=1)
            _, warped_grid = gmm(gmm_input, c_gmm); warped_c = F.grid_sample(c, warped_grid, padding_mode='border'); warped_cm = F.grid_sample(cm, warped_grid, padding_mode='border')
            misalign_mask = parse[:, 2:3] - warped_cm; misalign_mask[misalign_mask < 0.0] = 0.0
            parse_div = torch.cat((parse, misalign_mask), dim=1); parse_div[:, 2:3] -= misalign_mask
            output = alias(torch.cat((img_agnostic, pose, warped_c), dim=1), parse, parse_div, misalign_mask)
            unpaired_names = ['{}_{}'.format(img_name.split('_')[0], c_name) for img_name, c_name in zip(img_names, c_names)]
            save_images(output, unpaired_names, os.path.join(opt.save_dir, opt.name))
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu"); opt = get_opt(); print(opt)
    if not os.path.exists(os.path.join(opt.save_dir, opt.name)): os.makedirs(os.path.join(opt.save_dir, opt.name))
    seg = SegGenerator(opt, input_nc=opt.semantic_nc + 8, output_nc=opt.semantic_nc); gmm = GMM(opt, inputA_nc=7, inputB_nc=3)
    opt.semantic_nc = 7; alias = ALIASGenerator(opt, input_nc=9); opt.semantic_nc = 13
    load_checkpoint(seg, os.path.join(opt.checkpoint_dir, opt.seg_checkpoint)); load_checkpoint(gmm, os.path.join(opt.checkpoint_dir, opt.gmm_checkpoint)); load_checkpoint(alias, os.path.join(opt.checkpoint_dir, opt.alias_checkpoint))
    seg.to(device).eval(); gmm.to(device).eval(); alias.to(device).eval()
    test(opt, seg, gmm, alias, device)
if __name__ == '__main__': main()
"""
with open(TEST_PY_PATH, 'w') as f:
    f.write(corrected_test_py_content.replace('F.grid_sample(c, warped_grid, padding_mode=\'border\')', 'F.grid_sample(c, warped_grid, padding_mode=\'border\', align_corners=False)').replace('F.grid_sample(cm, warped_grid, padding_mode=\'border\')', 'F.grid_sample(cm, warped_grid, padding_mode=\'border\', align_corners=False)').replace("mode='bilinear'", "mode='bilinear', align_corners=False"))
print("✅ Setup Complete. Ready to start server.")

# --- SEGMENT 3: START THE SERVER ---
print("\n--- SEGMENT 3: STARTING THE FASTAPI SERVER ---")
app = FastAPI()
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_credentials=True, allow_methods=["*"], allow_headers=["*"])

@app.post("/run-inference/")
async def run_inference(person_image: UploadFile = File(...), cloth_image: UploadFile = File(...)):
    person_filename = person_image.filename
    cloth_filename = cloth_image.filename
    print(f"✅ Received request: Person='{person_filename}', Cloth='{cloth_filename}'")

    # 1. Save the uploaded files
    with open(os.path.join(VITON_HD_TEST_DIR, 'image', person_filename), "wb") as buffer: shutil.copyfileobj(person_image.file, buffer)
    with open(os.path.join(VITON_HD_TEST_DIR, 'cloth', cloth_filename), "wb") as buffer: shutil.copyfileobj(cloth_image.file, buffer)

    # 2. Overwrite test_pairs.txt
    pairs_file_path = os.path.join(VITON_HD_DATA_DIR, "test_pairs.txt")
    with open(pairs_file_path, 'w') as f: f.write(f"{person_filename} {cloth_filename}")
    print("✅ test_pairs.txt overwritten. Handing off to the model...")

    # 3. Run the inference commands
    try:
        # This now behaves exactly like your simple script, trusting the model's
        # internal dataloader to find the files in the folders you provided.
        subprocess.run(["python", "test.py", "--name", "alias_stage", "--dataset_list", "test_pairs.txt"], check=True, cwd=VITON_HD_CLONE_DIR, capture_output=True, text=True)
        print("✅ Inference complete.")
    except subprocess.CalledProcessError as e:
        # If it fails now, the error is from the model itself.
        print(f"❌ MODEL ERROR: {e.stderr}")
        raise HTTPException(status_code=500, detail=f"Model inference failed: {e.stderr}")

    # 4. Find and return the result image
    results_dir = os.path.join(VITON_HD_CLONE_DIR, 'results', 'alias_stage')
    person_id = person_filename.split('_')[0]
    expected_result_filename = f"{person_id}_{cloth_filename}"
    result_path = os.path.join(results_dir, expected_result_filename)
    if not os.path.exists(result_path):
        all_results = os.listdir(results_dir)
        if all_results: result_path = os.path.join(results_dir, all_results[0])
        else: raise HTTPException(status_code=404, detail="Result image not found.")

    return FileResponse(result_path, media_type="image/jpeg")

# --- SEGMENT 4: LAUNCH THE SERVER ---
ngrok_authtoken = "31D0MlOfXiyJnvVtIMmaYalNgjl_7mRbsniXVPb1c9CrNnm2J"
ngrok.set_auth_token(ngrok_authtoken)
http_tunnel = ngrok.connect(8000)
print("===================================================================")
print("                    ✅ Your Colab Backend is LIVE! ✅")
print(f"🔗 Public URL: {http_tunnel.public_url}")
print("\nACTION: Use this URL in your project's backend to send requests.")
print("===================================================================")
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

--- SEGMENT 0: INSTALLING LIBRARIES & IMPORTS ---

--- SEGMENT 1: SETTING UP THE ENVIRONMENT ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Copying base assets from: /content/drive/MyDrive/VirtualFIT_Models/viton_hd_zips

--- SEGMENT 2: PATCHING SCRIPT & INSTALLING DEPENDENCIES ---
✅ Setup Complete. Ready to start server.

--- SEGMENT 3: STARTING THE FASTAPI SERVER ---


INFO:     Started server process [23430]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


                    ✅ Your Colab Backend is LIVE! ✅
🔗 Public URL: https://208f3e471f84.ngrok-free.app

ACTION: Use this URL in your project's backend to send requests.
✅ Received request: Person='00067_00.jpg', Cloth='07118_00.jpg'
✅ test_pairs.txt overwritten. Handing off to the model...
✅ Inference complete.
INFO:     103.106.57.16:0 - "POST /run-inference/ HTTP/1.1" 200 OK
✅ Received request: Person='00112_00.jpg', Cloth='08255_00.jpg'
✅ test_pairs.txt overwritten. Handing off to the model...
✅ Inference complete.
INFO:     103.106.57.16:0 - "POST /run-inference/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23430]
